In [16]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import batman
import random
from astropy.io import ascii
from scipy import constants

In [17]:
#Define constants
M_sun = 1.989e+30
R_sun = 695700000
R_Earth = 6378100
day = 86400
AU = 1.496e+11

In [18]:
def planetmaker(number, rates, radius_low, radius_up, period_low, period_up):
    '''Generates parameters for a given number of planets in a planetary system from occurence rates'''
    rowdex = np.arange(len(rates))
    cosi = np.random.random_sample()
    anglew = np.random.uniform(low=-180., high=180., size=number)
    e = np.random.beta(a=1.03, b=13.6, size=number)
    rows = np.random.choice(rowdex, p=rates, size=number)
    radp = np.random.uniform(low=radius_low[rows], high=radius_up[rows], size=number)
    per = np.random.uniform(low=period_low[rows], high=period_up[rows], size=number)
    return radp, per, cosi, anglew, e

In [19]:
table = ascii.read('norm_rates_FGK.csv', format='csv', header_start=0, data_start=1)
occ = np.array(table['f'], dtype=float)
radbinlower = np.array(table['rmin'], dtype=float)
radbinupper = np.array(table['rmax'], dtype=float)
pbinlower = np.array(table['pmin'], dtype=float)
pbinupper = np.array(table['pmax'], dtype=float)

In [20]:
#Generate a random temperature. In the final pipeline Temperature will be chosen from TICv8
T_eff = np.random.uniform(3700.0, 7500.0)

In [21]:
#Assign spectral type based on temperature
if 7500<= T_eff <10000:
    spectral_type = 'A'
if 6000<= T_eff <7500:
    spectral_type = 'F'
if 5200<= T_eff <6000:
    spectral_type = 'G'
if 3700<= T_eff < 5200:
    spectral_type = 'K'
if 2400<= T_eff < 3700:
    spectral_type = 'M'

In [22]:
#Choose mass and radius dependent on spectral type. In the final pipeline these will come from TICv8
if spectral_type == 'A':
    M_star = np.random.uniform(2.1, 1.4)
    R_star = np.random.uniform(1.4, 1.8)
if spectral_type == 'F':
    M_star = np.random.uniform(1.04, 1.4)
    R_star = np.random.uniform(1.15, 1.4)
if spectral_type == 'G':
    M_star = np.random.uniform(0.8, 1.04)
    R_star = np.random.uniform(0.96, 1.15)
if spectral_type == 'K':
    M_star = np.random.uniform(0.45, 0.8)
    R_star = np.random.uniform(0.7, 0.96)
if spectral_type == 'M':
    M_star = np.random.uniform(0.08, 0.45)
    R_star = np.random.uniform(0.1, 0.8)

In [23]:
#Generate the number of planets in the system based on spectral type. These values of mu are temporary
if spectral_type in ['A', 'F', 'G', 'K']:
    n_planet = np.random.poisson(0.689)
if spectral_type == 'M':
    n_planet = np.random.poisson(2.5)

In [54]:
if spectral_type in ['F', 'G', 'K'] and n_planet > 0:
    radii, periods, cosi, peris, eccs = planetmaker(n_planet, occ, radbinlower, radbinupper, pbinlower, pbinupper)
    a = np.cbrt((constants.G*(M_star*M_sun)*np.square(periods*day))/(4*np.square(np.pi)))
    b = ((a*np.square(cosi))/(R_star*R_sun))*((1-np.square(eccs))/(1+eccs*np.sin(peris)))
    k = (radii*R_Earth)/(R_star*R_sun)
    Transit = (0<= b) & (b< 1)
    Grazing = (b >= 1) & (b < (1+k))
    inc = np.degrees(np.arccos(cosi))

In [59]:
if True in Transit:
    #Generate range of Time values to test for in/out transit times in TESS BJD, assuming a mission duration of 10 years and a cadence of 20s
    T_start = 0.0
    T_end = 3650.0
    cadence = 20.0
    T_range = np.arange(T_start, T_end, step=cadence/day, dtype=float)
    #Calculate Transit duration
    T_dur = (periods[Transit]*day/np.pi)*np.arcsin((R_star*R_sun/a)*np.sqrt(1+k[Transit]-np.square(b[Transit]))/np.sqrt(1-np.square(cosi)))
    T_0 = np.random.uniform(low=T_start, high=T_start+periods[Transit]) #First possible to observe transit ingress time
    T_1s = np.arange(T_0, T_end, periods[Transit]) #All transit ingress times over duration of TESS mission
    T_4s = T_1s + T_dur/day #All transit egress times over duration of TESS mission
    #In_Transit = (T_range > T_1s) & (T_range < T_4s)

In [56]:
print(spectral_type, 'type star')
print('Hosts',n_planet, 'planets')
if n_planet > 0:
    for i in range(n_planet):
        print('Planet', i)
        print('Radius = ',radii[i], 'Earth Radii')
        print('Period =', periods[i], 'days')
        print('Inclination angle =', incs, 'deg')
        print('Eccentricity =', eccs[i])
        print('Semi major axis =',a[i]/AU, 'AU')
        print('Impact parameter, b =', b[i])
        print('Transit?', Transit[i])
        print('Grazing?', Grazing[i])
        if Transit[i] == True or Grazing[i] == True:
            print('Transit duration =', T_dur[i]/3600, 'hours')
            print('Transits (ingress) occur at TBJD =', T_1s)

K type star
Hosts 1 planets
Planet 0
Radius =  1.0385631674217746 Earth Radii
Period = 7.098174399256717 days
Inclination angle = 51.677611951966384 deg
Eccentricity = 0.31589286810249023
Semi major axis = 0.06626834472202281 AU
Impact parameter, b = 0.001064421565333971
Transit? True
Grazing? False
Transit duration = 3.65379465790167 hours
Transits (ingress) occur at TBJD = [   6.00712819   13.10530259   20.20347699   27.30165139   34.39982579
   41.49800019   48.59617459   55.69434899   62.79252339   69.89069779
   76.98887219   84.08704658   91.18522098   98.28339538  105.38156978
  112.47974418  119.57791858  126.67609298  133.77426738  140.87244178
  147.97061618  155.06879058  162.16696498  169.26513938  176.36331377
  183.46148817  190.55966257  197.65783697  204.75601137  211.85418577
  218.95236017  226.05053457  233.14870897  240.24688337  247.34505777
  254.44323217  261.54140657  268.63958096  275.73775536  282.83592976
  289.93410416  297.03227856  304.13045296  311.228627